# Constraint scan

## Generate constraintscandf

In [1]:
import numpy as np
import pandas as pd
import pyDOE
from mmon_gcm.constraintscan import generate_constraints_df

## Define parameter bounds

In [2]:
parameter_bounds = {}

## Photons

We can go 10% above and below the value for photon reflectance and transmission from Zhu et al (2010)

In [3]:
parameter_bounds["P_abs"] = [0.9*0.9, 0.9*1.1, r"Dimensionless",
                             "90% of Zhu, Long, and Ort (2010)", "110% of Zhu, Long, and Ort (2010)"]  # proportion of photons absorbed by the leaf

The lower bound is the default we've used from Wuyts et al (2021), for a higher value we'll use one from Ramonell 2001.

In [4]:
parameter_bounds["T_l"] = [0.017 * 10 ** -2, 0.024 * 10 ** -2, r"m", "Wuyts et al. (2010)", "Ramonell et al. (2001)"]

The are of the leaf being simulated is fixed at 1m$^2$

In [5]:
parameter_bounds["A_l"] = [1, 1, r"m$^2$", "Fixed", "Fixed"]

For the lower bound we can use the average volume of an *Arabidopsis* guard cell as the midpoint of the values given in table 1 of Jezek and Blatt (2017). For the upper bound we can use the volume given in Hills et al (2012). 

In [6]:
j_b_upper = 0.65
j_b_lower = 0.3
V_gc_ind = (j_b_lower+j_b_upper)/2  # pL
V_gc_ind = V_gc_ind * 10**-12  # dm3
parameter_bounds["V_gc_ind"] = [V_gc_ind, 4.1E-12, r"dm$^3$", "Jezek and Blatt (2017)", "Hills et al. (2012)"]

The photosynthetic efficient of guard cells compared to the mesophyll was taken from Lawson (2003). 

> In leaves of all species the values of photosynthetic efficiency for guard cells were either indistinguishable from or only slightly lower (minimum of 79%) than those of the underlying, spongy mesophyll cells.

We'll use these as the bounds.

In [7]:
parameter_bounds["FqFm"] = [0.79, 0.9, r"Dimensionless", "Lawson (2003)", "Lawson (2003)"]

Fujiwara et al (2019) provide a range of values for number of chloroplasts in mesophyll and guard cells, we can take the upper gc and lower me and vice versa to get the upper and lower bound for the ratios, respectively.

In [8]:
upper_gc = 5.5
lower_gc = 3.5

upper_me = 100
lower_me = 30

rch_lower_bound = lower_gc/upper_me
rch_upper_bound = upper_gc/lower_me

parameter_bounds["R_ch"] = [rch_lower_bound, rch_upper_bound, r"Dimensionless",
                            "Fujiwara, Sanjaya, and Itoh (2019)", "Fujiwara, Sanjaya, and Itoh (2019)"]

Added in Nov 2023: The proportion of chloroplast volumes between guard cells and mesophyll cells for Arabidopsis Col-0 based on Chl a data from Knoblauch et al. (2023) Table S1. We can the standard error values to calculate the range with min/max being 2 SE away.

In [9]:
chl_vol_me = 88.24
chl_vol_se_me = 1.58
min_chl_vol_me = chl_vol_me - 2 * chl_vol_se_me
max_chl_vol_me = chl_vol_me + 2 * chl_vol_se_me

chl_vol_gc = 17.69
chl_vol_se_gc = 0.21
min_chl_vol_gc = chl_vol_gc - 2 * chl_vol_se_gc
max_chl_vol_gc = chl_vol_gc + 2 * chl_vol_se_gc

min_R_ch_vol = min_chl_vol_gc / max_chl_vol_me
max_R_ch_vol = max_chl_vol_gc / min_chl_vol_me

#modelparameters["R_ch_vol"] = [R_ch_vol, r"Dimensionless", "Knoblauch et al. (2023)"]

parameter_bounds["R_ch_vol"] = [min_R_ch_vol, max_R_ch_vol, r"Dimensionless",
                            "Knoblauch et al. (2023)", "Knoblauch et al. (2023)"]

For the lower bound Ramonell et al. (2001) provide a percentage of the leaf that is space at atmospheric pressure in Table 1.For the upper bound we take the proportion of the leaf that is air from Earles et al 2018 for *Guzmania zahnii* from Table 2. 

In [10]:
parameter_bounds["L_air"] = [0.185, 0.37, r"Dimensionless", "Ramonell et al. (2001)", "Earles et al. (2018)"]

Willmer and Fricker (1996) provide a lower bound of 0.1 and Ramonell et al. (2001) an upper bound of 0.24 for the proportion of the leaf that is epidermis.

In [11]:
parameter_bounds["L_epidermis"] = [0.1, 0.24, r"Dimensionless", "Willmer and Fricker (1996)", "Ramonell et al. (2001)"]

## Osmolarity

Wang et al. (2017) use a value of 0.751 to be the proportion of the Guard Cell that is vacuole, Andrés et al. (2014) quote a value of 90% from an old MacRobbie paper.  

In [12]:
parameter_bounds["Vac_frac"] = [0.751, 0.9, r"Dimensionless", "Wang et al. (2017)", "Andrés et al. (2014) "]

A reasonable range for temperature seems to be 10C to 25C, this covers a wide range:

In [13]:
parameter_bounds["T"] = [273.15+10, 273.15+25, "K", "10C", "25C"]

Ideal gas constant, from NIST, (reference Tiesinga2019). This is constant, so won't change in the model

In [14]:
R = 8.205*10**(-5)  # m3atmK-1mol-1
R = R*10**3  # dm3atmK-1mol-1
parameter_bounds["R"] = [
    R, R, r"dm$^3$$\cdot$atm$\cdot$K$^{-1}$$\cdot$mol$^{-1}$", "Tiesinga et al. (2019)", "Tiesinga et al. (2019)"]

The density of guard cells per m$^2$ of leaf was taken by multiplying the density of stomata from Papanatsiou et al (2016) by two. This only takes into account the abaxial surface of the leaf, so for the upper bound, we'll double this again. For the lower bound, we'll use a value for *Commelina* from Willmer and Fricker (1996). 

In [15]:
parameter_bounds["N_gcs"] = [172 * 10 ** 6, 290 * 10 ** 6 * 2.0 * 2,
                             r"GCs$\cdot$m$^{-2}$", "Willmer and Fricker (1996)", "Papanatsiou, Amtmann, and Blatt (2016)"]

The following parameters required for osmolarity are based taking values from an older OnGuard paper (Wang et al. (2012)) and the newer, updated model (Wang et al 2017).

In [16]:
parameter_bounds["n"] = [1.5, 2.5, "atm", "Wang et al. (2012)", "Wang et al. (2017)"]
parameter_bounds["m"] = [0.8, 1, r"atm$\cdot$µm$^{-1}$", "Wang et al. (2017)", "Wang et al. (2012)"]
parameter_bounds["r"] = [0.05 * 10 ** (-12), 0.08 * 10 ** (-12),
                         r"dm$^3$ µm$^{-1}$", "Wang et al. (2017)", "Wang et al. (2012)"]
parameter_bounds["s"] = [0.1 * 10 ** (-12), 0.3 * 10 ** (-12), r"dm$^3$", "Wang et al. (2012)", "Wang et al. (2017)"]

For the apoplastic concentration, we use the value from the OnGuard model as the lower bound, and use a combination of values which have been collated by Roelfsema and Hedrich (2002) from Lohaus et al. (2001) for the upper bound:

In [17]:
apoplastic_concs_wang_2017 = {
    "K": 10,
    "Ca": 1,
    "Cl": 12,
    "Suc": 0.01,
    "MH2": 3.2*10**(-6),
    "MH": 7.9*10**(-5),
    "M": 0.00999,
}
apoplastic_conc_wang_2017 = sum(apoplastic_concs_wang_2017.values())

In [18]:
apoplastic_concs_roelfsema_2002 = {
    "K": 13,
    "Ca": 0.7,
    "Cl": 11,
    "Suc": 1.6,
    "Mal": 0.7,
    "Amino Acids": 9.6,
    "Hexoses": 0.7,
}
apoplastic_conc_roelfsema_2002 = sum(apoplastic_concs_roelfsema_2002.values())

In [19]:
parameter_bounds["C_apo"] = [apoplastic_conc_wang_2017*10 **
                             (-3), apoplastic_conc_roelfsema_2002*10**(-3), r"mol$\cdot$dm$^{-3}$", "Wang et al. (2017)", "Roelfsema and Hedrich (2002)"]

## Apertures

For the closed aperture, we'll use 1 µm value from Jezek and Blatt (2017) as a lower bound and 4 µm from Wang et al. (2017) as an upper bound.

In [20]:
parameter_bounds["A_closed"] = [1, 4, r"µm", "Jezek and Blatt (2017)", "Wang et al. (2017)"]

For the open aperture, we'll use the 2.75 µm from Horrer et al. (2016) as the lower bound and 12 µm from Wang et al. (2017) as the upper bound.

In [21]:
parameter_bounds["A_open"] = [2.75, 12, r"µm", "Horrer et al. (2016)", "Wang et al. (2017)"]

For the proportion of photons that are absorbed by the leaf we can just go for 10% above and below the value that is reported by 

## ATPase

We'll go between constraining the ATPase to 0 and the level of ATPase that Flütsch et al. (2020) measured (17 fmoles$\cdot$GC$^{-1}\cdot$h$^{-1}$).

In [22]:
parameter_bounds["ATPase"] = [
    0, 17, r"fmoles$\cdot$GC$^{-1}\cdot$h$^{-1}$", "Supplementary", "Flütsch et al. (2020)"]

## Maintenance

We test between 0 and 0.003 for the proportion of dark maintenance contributed by GC

In [23]:
parameter_bounds["Maintenance"] = [
    0, 0.003, r"fmoles$\cdot$GC$^{-1}\cdot$h$^{-1}$", "0 (no maintenance)", "Based on GC volume"]

# Export parameters to csv

In [24]:
params_df = pd.DataFrame.from_dict(parameter_bounds, orient="index", columns=[
                                   "Lower", "Upper", "Units", "Source Lower", "Source Upper"])
params_df.to_csv("../outputs/constraint_scan/parameter_bounds.csv")
params_df.to_csv("../plant_cell_paper/supplemental_table_2.csv")

In [25]:
params_df

,Lower,Upper,Units,Source Lower,Source Upper
P_abs,8.100000e-01,9.900000e-01,Dimensionless,"90% of Zhu, Long, and Ort (2010)","110% of Zhu, Long, and Ort (2010)"
T_l,1.700000e-04,2.400000e-04,m,Wuyts et al. (2010),Ramonell et al. (2001)
A_l,1.000000e+00,1.000000e+00,m$^2$,Fixed,Fixed
V_gc_ind,4.750000e-13,4.100000e-12,dm$^3$,Jezek and Blatt (2017),Hills et al. (2012)
FqFm,7.900000e-01,9.000000e-01,Dimensionless,Lawson (2003),Lawson (2003)
R_ch,3.500000e-02,1.833333e-01,Dimensionless,"Fujiwara, Sanjaya, and Itoh (2019)","Fujiwara, Sanjaya, and Itoh (2019)"
R_ch_vol,1.889497e-01,2.128585e-01,Dimensionless,Knoblauch et al. (2023),Knoblauch et al. (2023)
L_air,1.850000e-01,3.700000e-01,Dimensionless,Ramonell et al. (2001),Earles et al. (2018)
L_epidermis,1.000000e-01,2.400000e-01,Dimensionless,Willmer and Fricker (1996),Ramonell et al. (2001)
Vac_frac,7.510000e-01,9.000000e-01,Dimensionless,Wang et al. (2017),Andrés et al. (2014)


### Generate constraints array

In [26]:
np.random.seed(12345)
lhs_df = pd.DataFrame(pyDOE.lhs(len(params_df), samples=1000), columns=params_df.index)

In [27]:
lhs_df

,P_abs,T_l,A_l,V_gc_ind,FqFm,R_ch,R_ch_vol,L_air,L_epidermis,Vac_frac,...,N_gcs,n,m,r,s,C_apo,A_closed,A_open,ATPase,Maintenance
0,0.062573,0.699974,0.395941,0.416698,0.940356,0.874637,0.939212,0.013475,0.603577,0.970953,...,0.070422,0.991444,0.545612,0.118471,0.648137,0.064375,0.434343,0.291074,0.527047,0.271748
1,0.729845,0.119321,0.297931,0.861780,0.986407,0.808892,0.883628,0.881991,0.787886,0.461443,...,0.552169,0.659945,0.409469,0.473439,0.583276,0.170806,0.852512,0.913360,0.379497,0.963707
2,0.928471,0.042669,0.098064,0.652606,0.486197,0.906728,0.901615,0.004911,0.310122,0.261830,...,0.755784,0.701635,0.968917,0.242032,0.142481,0.882029,0.202425,0.231836,0.054463,0.218171
3,0.151511,0.663287,0.642178,0.637382,0.436228,0.275392,0.999258,0.112016,0.509757,0.156989,...,0.641742,0.386525,0.886661,0.354706,0.306482,0.109805,0.178519,0.057578,0.427713,0.437904
4,0.633595,0.140870,0.361887,0.671450,0.189334,0.980359,0.933710,0.794987,0.149721,0.912310,...,0.344479,0.461033,0.644283,0.173250,0.978940,0.841682,0.787571,0.968603,0.800916,0.371208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.904837,0.287000,0.884344,0.842649,0.035380,0.792938,0.534630,0.849561,0.896163,0.400245,...,0.471860,0.523977,0.298841,0.443951,0.414121,0.161314,0.307774,0.922045,0.743934,0.830516
996,0.618690,0.332661,0.317465,0.209934,0.275060,0.514937,0.276113,0.324327,0.225151,0.148337,...,0.237827,0.505273,0.292813,0.430939,0.896529,0.092123,0.259698,0.622974,0.456745,0.019688
997,0.521309,0.252984,0.581883,0.450729,0.502747,0.159051,0.114418,0.249752,0.109324,0.809865,...,0.308602,0.011255,0.340746,0.693372,0.768127,0.894854,0.074700,0.885657,0.599506,0.115667
998,0.263881,0.793845,0.348736,0.683502,0.837938,0.911102,0.637290,0.219920,0.477385,0.839055,...,0.416364,0.275285,0.592821,0.572289,0.128359,0.012682,0.199993,0.720383,0.680544,0.703183


In [28]:
constraints_df = generate_constraints_df(lhs_df, params_df)
constraints_df.head()

Removing 35 combinations where open aperture is smaller than closed


,P_abs,T_l,A_l,V_gc_ind,FqFm,R_ch,R_ch_vol,L_air,L_epidermis,Vac_frac,...,N_gcs,n,m,r,s,C_apo,A_closed,A_open,ATPase,Maintenance
0,0.821263,0.000219,1.0,1.985531e-12,0.893439,0.164738,0.211405,0.187493,0.184501,0.895672,...,2.415771e+08,2.491444,0.909122,5.355413e-14,2.296275e-13,0.023939,2.303029,5.442436,8.959799,0.000815
1,0.941372,0.000178,1.0,3.598954e-12,0.898505,0.154986,0.210076,0.348168,0.210304,0.819755,...,7.175430e+08,2.159945,0.881894,6.420317e-14,2.166552e-13,0.025459,3.557535,11.198578,6.451445,0.002891
2,0.977125,0.000173,1.0,2.840697e-12,0.843482,0.169498,0.210506,0.185908,0.143417,0.790013,...,9.187141e+08,2.201635,0.993783,5.726095e-14,1.284962e-13,0.035615,1.607276,4.894487,0.925869,0.000655
3,0.837272,0.000216,1.0,2.785511e-12,0.837985,0.075850,0.212841,0.205723,0.171366,0.774391,...,8.060406e+08,1.886525,0.977332,6.064119e-14,1.612963e-13,0.024588,1.535557,3.282600,7.271118,0.001314
4,0.924047,0.000180,1.0,2.909005e-12,0.810827,0.180420,0.211274,0.332073,0.120961,0.886934,...,5.123453e+08,1.961033,0.928857,5.519750e-14,2.957881e-13,0.035039,3.362713,11.709579,13.615575,0.001114


In [29]:
constraints_df.to_csv("../outputs/constraint_scan/constraints_df.csv")